In [1]:
import imageio
import numpy as np

In [2]:
grayscale = imageio.imread('Bayesnoise_grayscale.png')
actual = imageio.imread('Bayes_grayscale.png')
bayes_textbook = imageio.imread('Bayes_textbook.png')

In [3]:
grayscale=np.where(grayscale==255, 1, grayscale) 
grayscale=np.where(grayscale==0, -1, grayscale) 

In [4]:
x=np.copy(grayscale)
y=np.copy(grayscale)

In [5]:
def get_neighbours(x,r,c):
    if c!=0:
        left=x[r][c-1]
    else:
        left=0
    if c!=len(x[0])-1:
        right=x[r][c+1]
    else:
        right=0
    if r!=0:
        up=x[r-1][c]
    else:
        up=0
    if r!=len(x)-1:
        down=x[r+1][c]
    else:
        down=0
    return left,right,up,down
    

In [6]:
def get_energy(h,beta,eta,me,left,right,up,down,other):
    energy=h*me -beta*(me*left + me*right + me*up + me*down) - eta*(me*other)
    return energy

In [7]:
def optimize(x,y,h,beta,eta):
    energy=0
    updated_x=np.copy(x)
    for r in range(len(x)):
        for c in range(len(x[0])):
            me=x[r][c]
            left,right,up,down=get_neighbours(x,r,c)
            other=y[r][c]
            negative_me=me*-1
            my_energy=get_energy(h,beta,eta,me,left,right,up,down,other)
            negative_me_energy=get_energy(h,beta,eta,negative_me,left,right,up,down,other)
            if negative_me_energy < my_energy:
                energy+=negative_me_energy
                updated_x[r][c]=negative_me
            else:
                energy+=my_energy
    return updated_x,energy

In [8]:
def coordinate_descent(x,y,h,beta,eta,max_iter):
    working_x=np.copy(x)
    c=0
    working_x,min_energy=optimize(working_x,y,h,beta,eta)
    while c<max_iter:
        working_x,energy=optimize(working_x,y,h,beta,eta)
        c+=1
        if energy<min_energy:
            min_energy=energy
        elif energy==min_energy:
            break
    return working_x        

In [9]:
def check_accuracy(predict,actual):
    correct=0
    for r in range(len(predict)):
        for c in range(len(predict[0])):
            if predict[r][c]==actual[r][c]:
                correct+=1
    return correct/(actual.shape[0]*actual.shape[1])

In [10]:
optimized=coordinate_descent(x,y,0.1,0.7,1,50)
optimized=np.where(optimized==1, 255, optimized) 
optimized=np.where(optimized==-1, 0, optimized)
accuracy=check_accuracy(optimized,actual)
imageio.imwrite("result.png", optimized)
print(accuracy)

0.9937375593324146


In [11]:
import visvis as vv
im = imageio.imread('result.png')
vv.imshow(im)

In [12]:
bw=np.copy(optimized)
def color(bw):
    colored=np.empty((311, 420, 4))
    for r in range(len(bw)):
        for c in range(len(bw[0])):
            if bw[r][c]==0:
                colored[r][c]=[0, 0, 255, 255]
            else:
                colored[r][c]=[247, 211, 109, 255]
    imageio.imwrite("cleaned.png", colored)
    return colored

clean=color(bw)    

In [13]:
def check_colored_accuracy(predict,actual):
    correct=0
    for r in range(len(predict)):
        for c in range(len(predict[0])):
            if predict[r][c].all()==actual[r][c].all():
                correct+=1
    return correct/(actual.shape[0]*actual.shape[1])
check_colored_accuracy(clean,bayes_textbook)

0.9937375593324146